In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns



In [15]:
# unpickle trips.pkl
trips = pd.read_pickle('trips_1M.pkl')
print(len(trips))

1043


In [20]:

trip = trips[50]

trip_id = trip[0]
trip_X = trip[1]
trip_y = trip[2]
trip_y_pred = trip[3]


# consider features trip_X as 'AbsoluteTime','Latitude', 'Longitude','Tsurf', 'Ta','Hours','Speed'

# convert trip_X to dataframe
trip_X = pd.DataFrame(trip_X, columns=['AbsoluteTime','Latitude', 'Longitude','Tsurf', 'Ta','Hours','Speed'])


